In [1]:
%pip install pandas numpy mutagen tidalapi requests --quiet


Note: you may need to restart the kernel to use updated packages.


# Local MP3 to Tidal Lookup

This notebook scans a local directory for MP3 files and searches for each song on Tidal.

**Purpose**: Find local music files in Tidal's catalog for potential migration or catalog comparison

**Requirements**:
- Local MP3 files with proper metadata
- Tidal API access (you'll need credentials or tidalapi library)
- Python libraries: mutagen, tidalapi, pandas


In [2]:
import os
import json
import pandas as pd
from pathlib import Path
from typing import List, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# Audio metadata
from mutagen.mp3 import MP3
from mutagen.easyid3 import EasyID3

print("✓ Core imports loaded successfully")


✓ Core imports loaded successfully


In [3]:
# Tidal API - Choose one method below

# Method 1: Using tidalapi library (recommended)
try:
    import tidalapi
    TIDAL_METHOD = "tidalapi"
    print("✓ tidalapi available")
except ImportError:
    print("⚠ tidalapi not installed. Install with: pip install tidalapi")
    TIDAL_METHOD = None

# Method 2: Manual API calls (fallback)
import requests
print("✓ requests available for manual API calls")


✓ tidalapi available
✓ requests available for manual API calls


In [4]:
def extract_mp3_metadata(file_path: str) -> Optional[Dict]:
    """
    Extract artist, title, album from MP3 file metadata
    
    Args:
        file_path: Path to MP3 file
        
    Returns:
        Dict with title, artist, album, or None if extraction fails
    """
    try:
        # Try ID3 tags first
        audio = EasyID3(file_path)
        metadata = {
            'title': audio.get('title', ['Unknown'])[0] if 'title' in audio else 'Unknown',
            'artist': audio.get('artist', ['Unknown'])[0] if 'artist' in audio else 'Unknown',
            'album': audio.get('album', ['Unknown'])[0] if 'album' in audio else 'Unknown',
            'path': file_path
        }
        return metadata
    except Exception as e:
        print(f"⚠ Could not read metadata from {file_path}: {str(e)}")
        return None


def scan_directory_for_mp3s(directory: str, recursive: bool = True) -> List[Dict]:
    """
    Scan directory for MP3 files and extract metadata
    
    Args:
        directory: Path to directory to scan
        recursive: Whether to search subdirectories
        
    Returns:
        List of dicts with MP3 metadata
    """
    mp3_files = []
    directory_path = Path(directory)
    
    if not directory_path.exists():
        print(f"✗ Directory not found: {directory}")
        return []
    
    # Get all MP3 files
    if recursive:
        pattern = "**/*.mp3"
    else:
        pattern = "*.mp3"
    
    mp3_paths = list(directory_path.glob(pattern))
    print(f"Found {len(mp3_paths)} MP3 files")
    
    for mp3_path in mp3_paths:
        metadata = extract_mp3_metadata(str(mp3_path))
        if metadata:
            mp3_files.append(metadata)
    
    return mp3_files


# Test the function
print("✓ MP3 scanning functions defined")


✓ MP3 scanning functions defined


In [5]:
def search_tidal_tidalapi(session, title: str, artist: str) -> Optional[Dict]:
    """
    Search Tidal using tidalapi library
    
    Args:
        session: tidalapi session object
        title: Song title
        artist: Artist name
        
    Returns:
        Dict with Tidal match info or None
    """
    try:
        # Search for track
        search_term = f"{title} {artist}"
        results = session.search(search_term, models=[tidalapi.Track])
        
        if results and results['tracks']:
            track = results['tracks'][0]
            return {
                'tidal_track_id': track.id,
                'tidal_title': track.name,
                'tidal_artist': track.artist.name if track.artist else 'Unknown',
                'tidal_album': track.album.name if track.album else 'Unknown',
                'found': True
            }
    except Exception as e:
        print(f"Error searching Tidal: {str(e)}")
    
    return {'found': False}


def search_tidal_manual(title: str, artist: str, country_code: str = "US") -> Optional[Dict]:
    """
    Search Tidal using public API (no auth required for basic search)
    
    Args:
        title: Song title
        artist: Artist name
        country_code: Country code (default: US)
        
    Returns:
        Dict with Tidal match info or None
    """
    try:
        url = "https://api.tidalhifi.com/v1/search/tracks"
        params = {
            'query': f"{title} {artist}",
            'limit': 1,
            'countryCode': country_code
        }
        
        response = requests.get(url, params=params, timeout=5)
        if response.status_code == 200:
            data = response.json()
            if data.get('items'):
                track = data['items'][0]
                return {
                    'tidal_track_id': track.get('id'),
                    'tidal_title': track.get('title'),
                    'tidal_artist': track.get('artist', {}).get('name', 'Unknown'),
                    'tidal_album': track.get('album', {}).get('title', 'Unknown'),
                    'found': True
                }
    except Exception as e:
        print(f"Error searching Tidal: {str(e)}")
    
    return {'found': False}


print("✓ Tidal search functions defined")


✓ Tidal search functions defined


In [6]:
def match_local_to_tidal(mp3_files: List[Dict], session=None, use_manual: bool = False) -> pd.DataFrame:
    """
    Match local MP3 files to Tidal catalog
    
    Args:
        mp3_files: List of MP3 metadata dicts from scan_directory_for_mp3s()
        session: tidalapi session (if using tidalapi method)
        use_manual: Force use of manual API search instead of tidalapi
        
    Returns:
        DataFrame with results
    """
    results = []
    
    for i, mp3 in enumerate(mp3_files, 1):
        print(f"[{i}/{len(mp3_files)}] Searching for: {mp3['artist']} - {mp3['title']}", end=" ... ")
        
        # Choose search method
        if not use_manual and session is not None and TIDAL_METHOD == "tidalapi":
            tidal_result = search_tidal_tidalapi(session, mp3['title'], mp3['artist'])
        else:
            tidal_result = search_tidal_manual(mp3['title'], mp3['artist'])
        
        # Combine results
        combined = {
            'local_artist': mp3['artist'],
            'local_title': mp3['title'],
            'local_album': mp3['album'],
            'local_path': mp3['path'],
            **tidal_result
        }
        
        status = "✓ Found" if tidal_result.get('found') else "✗ Not found"
        print(status)
        
        results.append(combined)
    
    df = pd.DataFrame(results)
    return df


print("✓ Main orchestration function defined")


✓ Main orchestration function defined


## Configuration

Update these settings before running:


In [7]:
# ===== CONFIGURATION =====

# 1. LOCAL DIRECTORY TO SCAN
# Use one of these formats:
#   - Home directory: "~/Music/Music_mp3s"
#   - Relative path: "Music/Music_mp3s"
#   - Full path: "/Users/maxfiep/Library/CloudStorage/.../Music_mp3s"
LOCAL_MUSIC_DIRECTORY = os.path.expanduser("~/Library/CloudStorage/GoogleDrive-pmaxfield@gmail.com/My Drive/Music/Music_mp3s")

# 2. TIDAL SETTINGS
TIDAL_USE_MANUAL_API = True  # Set to False if using tidalapi with auth
TIDAL_COUNTRY_CODE = "US"

# 3. OPTIONAL: Tidal authentication (if using tidalapi)
# TIDAL_USERNAME = "your_tidal_username"
# TIDAL_PASSWORD = "your_tidal_password"

# ===== END CONFIGURATION =====

print(f"Music directory: {LOCAL_MUSIC_DIRECTORY}")
print(f"Tidal method: {'Manual API' if TIDAL_USE_MANUAL_API else 'tidalapi'}")
print(f"Country code: {TIDAL_COUNTRY_CODE}")


Music directory: /Users/maxfiep/Library/CloudStorage/GoogleDrive-pmaxfield@gmail.com/My Drive/Music/Music_mp3s
Tidal method: Manual API
Country code: US


## Step 1: Scan Local Directory


In [8]:
# Scan the directory for MP3 files
mp3_metadata = scan_directory_for_mp3s(LOCAL_MUSIC_DIRECTORY, recursive=True)

print(f"\n✓ Scanned complete")
print(f"Total MP3 files found: {len(mp3_metadata)}")

# Display sample of files found (show only the filename, not the full path)
if mp3_metadata:
    df_preview = pd.DataFrame(mp3_metadata).head(10)
    print("\nSample of files found:")
    if "filepath" in df_preview.columns:
        # Show only the filename from the filepath column
        print(df_preview['filepath'].apply(lambda x: os.path.basename(x)).to_string(index=False, header=["filename"]))
    else:
        print(df_preview.to_string(index=False))
else:
    print("\n⚠ No MP3 files found. Check the LOCAL_MUSIC_DIRECTORY path.")


Found 56 MP3 files
⚠ Could not read metadata from /Users/maxfiep/Library/CloudStorage/GoogleDrive-pmaxfield@gmail.com/My Drive/Music/Music_mp3s/Janes Addiction/Jane's Addiction - Three Days - Hammerstein Ballroom 97.mp3: "/Users/maxfiep/Library/CloudStorage/GoogleDrive-pmaxfield@gmail.com/My Drive/Music/Music_mp3s/Janes Addiction/Jane's Addiction - Three Days - Hammerstein Ballroom 97.mp3" doesn't start with an ID3 tag

✓ Scanned complete
Total MP3 files found: 55

Sample of files found:
                                           title                               artist                                    album                                                                                                                                                                                                         path
                Old Saltillo Road (Instrumental)                       The Heavy Eyes                       He Dreams of Lions                                          /Users/

## Step 2: Search Tidal for Matches

This step will search each local MP3 on Tidal's catalog.


In [9]:
# Initialize Tidal session if using tidalapi
tidal_session = None
if not TIDAL_USE_MANUAL_API and TIDAL_METHOD == "tidalapi":
    try:
        tidal_session = tidalapi.Session()
        # Uncomment below if you want to authenticate with credentials
        # tidal_session.login(TIDAL_USERNAME, TIDAL_PASSWORD)
        print("✓ Tidal session initialized")
    except Exception as e:
        print(f"⚠ Could not initialize Tidal session: {e}")
        print("  Falling back to manual API...")
        TIDAL_USE_MANUAL_API = True

# Perform matching
if mp3_metadata:
    results_df = match_local_to_tidal(mp3_metadata, session=tidal_session, use_manual=TIDAL_USE_MANUAL_API)
    print(f"\n✓ Matching complete")
else:
    print("✗ No MP3 files to search. Please check the directory in Step 1.")


[1/55] Searching for: The Heavy Eyes - Old Saltillo Road (Instrumental) ... ✗ Not found
[2/55] Searching for: Eagles of Death Metal - Complexity ... ✗ Not found
[3/55] Searching for: Eagles of Death Metal - Anything 'Cept the Truth (live Le Trianon Paris) ... ✗ Not found
[4/55] Searching for: Dave Grohl, Josh Homme, Trent Reznor - Mantra (Instrumental) ... ✗ Not found
[5/55] Searching for: Them Crooked Vultures - Long Slow Goodbye ... ✗ Not found
[6/55] Searching for: Them Crooked Vultures - Gunman ... ✗ Not found
[7/55] Searching for: Them Crooked Vultures - Spinning in Daffodils (Fuji Rock 2010) trim ... ✗ Not found
[8/55] Searching for: Jack White & Loretta Lynn - Portland Oregon ... ✗ Not found
[9/55] Searching for: The Doors w Ian Astbury - Love Me Two Times ... ✗ Not found
[10/55] Searching for: Ed Rush & Optical - Funktion (remix) ... ✗ Not found
[11/55] Searching for: Doors, The - Break On Through w/ Ian Astbury ... ✗ Not found
[12/55] Searching for: Desert Sessions - Nenada ..

## Step 3: View Results


In [10]:
# Summary statistics
if 'results_df' in locals():
    found_count = results_df['found'].sum()
    total_count = len(results_df)
    match_percentage = (found_count / total_count * 100) if total_count > 0 else 0
    
    print("=" * 60)
    print("RESULTS SUMMARY")
    print("=" * 60)
    print(f"Total songs:        {total_count}")
    print(f"Found on Tidal:     {found_count}")
    print(f"Not found:          {total_count - found_count}")
    print(f"Match rate:         {match_percentage:.1f}%")
    print("=" * 60)


RESULTS SUMMARY
Total songs:        55
Found on Tidal:     0
Not found:          55
Match rate:         0.0%


In [11]:
# Display full results table
if 'results_df' in locals():
    print("\nFull Results (first 20 rows):")
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    print(results_df.head(20).to_string(index=False))



Full Results (first 20 rows):
                        local_artist                                      local_title                              local_album                                                                                                                                                                                                               local_path  found
                      The Heavy Eyes                 Old Saltillo Road (Instrumental)                       He Dreams of Lions                                                      /Users/maxfiep/Library/CloudStorage/GoogleDrive-pmaxfield@gmail.com/My Drive/Music/Music_mp3s/The Heavy Eyes/Old Saltillo Road (Instrumental) by The Heavy Eyes.mp3  False
               Eagles of Death Metal                                       Complexity                              Zipper Down                                                 /Users/maxfiep/Library/CloudStorage/GoogleDrive-pmaxfield@gmail.com/My Drive/Music/Music_mp3

## Step 4: Export Results
